# ETL Project

## Comparison of closing price between cryptocurrencies and ETFs

#### Adan Bonilla, Morgan Ivey & Orlando Lepe

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from databaseconfig import user_name, password, local_host
import datetime

## Extract CSVs and TXTs into DataFrames

1. Cryptocurrency Data : [Cryptocurrency Source](
https://www.kaggle.com/philmohun/cryptocurrency-financial-data)

2. Stock Data : [Stock Data Source](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs#aadr.us.txt)

In [5]:
# Import crypto CSV data
crypto_data = pd.read_csv('Resources/consolidated_coin_data.csv')
crypto_df = crypto_data.copy()

# Convert to Date datetime object
crypto_df['Date']=pd.to_datetime(crypto_df['Date'])

# Find the first and last dates in dataset
first_date_crypto = crypto_df['Date'].iloc[0].strftime('%Y-%m-%d')
last_date_crypto = crypto_df['Date'].iloc[len(crypto_df)-1].strftime('%Y-%m-%d')

# Preview DataFrame
crypto_df.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,tezos,2019-12-04,1.29,1.32,1.25,1.25,"46,048,752","824,588,509"
1,tezos,2019-12-03,1.24,1.32,1.21,1.29,"41,462,224","853,213,342"
2,tezos,2019-12-02,1.25,1.26,1.20,1.24,"27,574,097","817,872,179"
3,tezos,2019-12-01,1.33,1.34,1.25,1.25,"24,127,567","828,296,390"
4,tezos,2019-11-30,1.31,1.37,1.31,1.33,"28,706,667","879,181,680"


In [18]:
# Extract the list of file names (<etf>.us.txt)
file_names = os.listdir('Resources/price-volume-data-for-all-us-stocks-etfs/ETFs')
file_names

['vti.us.txt',
 'ibnd.us.txt',
 'viov.us.txt',
 'urty.us.txt',
 'schr.us.txt',
 'jnk.us.txt',
 'pej.us.txt',
 'eqwm.us.txt',
 'rusl.us.txt',
 'feny.us.txt',
 'lglv.us.txt',
 'don.us.txt',
 'dim.us.txt',
 'vfh.us.txt',
 'vug.us.txt',
 'faus.us.txt',
 'eelv.us.txt',
 'vqt.us.txt',
 'xt.us.txt',
 'ujb.us.txt',
 'iwm.us.txt',
 'uym.us.txt',
 'fieu.us.txt',
 'ashs.us.txt',
 'chim.us.txt',
 'dia.us.txt',
 'xtn.us.txt',
 'agg.us.txt',
 'tip.us.txt',
 'gbf.us.txt',
 'fvd.us.txt',
 'rvnu.us.txt',
 'pst.us.txt',
 'smlf.us.txt',
 'dnl.us.txt',
 'know.us.txt',
 'sil.us.txt',
 'dxjf.us.txt',
 'puw.us.txt',
 'usci.us.txt',
 'mxi.us.txt',
 'jjt.us.txt',
 'gsy.us.txt',
 'hevy.us.txt',
 'pvi.us.txt',
 'sqqq.us.txt',
 'iwy.us.txt',
 'hdv.us.txt',
 'gnma.us.txt',
 'flot.us.txt',
 'idv.us.txt',
 'sdow.us.txt',
 'ldri.us.txt',
 'midu.us.txt',
 'schf.us.txt',
 'mom.us.txt',
 'sye.us.txt',
 'prb.us.txt',
 'moat.us.txt',
 'rye.us.txt',
 'iak.us.txt',
 'tok.us.txt',
 'ivw.us.txt',
 'llsp.us.txt',
 'eld.us.txt'

In [19]:
# Create empty DataFrame (main DataFrame for ETFs) 
all_stocks = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume','OpenInt','ETF'])

# Append each .txt file to main DataFrame, account for/skip empty files
for file in file_names:
    if os.stat(f'Resources/price-volume-data-for-all-us-stocks-etfs/ETFs/{file}').st_size > 0:
        stock_data = pd.read_csv(f'Resources/price-volume-data-for-all-us-stocks-etfs/ETFs/{file}')
        stock_data['ETF'] = file.split('.')[0]
        stock_data = stock_data[stock_data['Date']>=last_date_crypto]
        all_stocks = all_stocks.append(stock_data)
    else:
        print(f'Empty File : {file}')

# Export stock DataFrame to csv file 
all_stocks.to_csv('Resources/all_etf_data.csv')

In [20]:
# Preview all_stocks DataFrame
all_stocks.head()

,Date,Open,High,Low,Close,Volume,OpenInt,ETF
2055,2013-04-29,74.952,75.393,74.842,75.227,2309512,0,vti
2056,2013-04-30,75.242,75.488,74.936,75.452,1897648,0,vti
2057,2013-05-01,75.319,75.375,74.649,74.688,2354331,0,vti
2058,2013-05-02,74.861,75.511,74.861,75.466,1761879,0,vti
2059,2013-05-03,76.127,76.484,76.006,76.172,2597651,0,vti


In [2]:
# Extract the Stock CSV
all_stock_df = pd.read_csv('Resources/all_etf_data.csv')
all_stock_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,OpenInt,ETF
0,2055,2013-04-29,74.952,75.393,74.842,75.227,2309512,0,vti
1,2056,2013-04-30,75.242,75.488,74.936,75.452,1897648,0,vti
2,2057,2013-05-01,75.319,75.375,74.649,74.688,2354331,0,vti
3,2058,2013-05-02,74.861,75.511,74.861,75.466,1761879,0,vti
4,2059,2013-05-03,76.127,76.484,76.006,76.172,2597651,0,vti


## Transform the ETF DataFrame

In [3]:
# Change the date column to datetime (format %Y-%m-%d)
all_stock_df['Date'] = pd.to_datetime(all_stock_df['Date'])

## Transform the Cryptocurrency DataFrame

In [6]:
# Convert date to match ETF's datetime (format %Y-%m-%d)
crypto_df['Date'] = pd.to_datetime(crypto_df['Date'])

# Sort date values by Date
crypto_df = crypto_df.sort_values(by=["Date"])

# Preview cryptocurrency DataFrame
crypto_df.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
14471,xrp,2013-04-28,135.30,135.98,132.10,134.21,0,"1,488,566,728"
26531,cardano,2013-04-28,4.30,4.40,4.18,4.35,0,"74,636,938"
24119,ethereum,2013-04-28,135.30,135.98,132.10,134.21,0,"1,488,566,728"
21707,litecoin,2013-04-28,4.30,4.40,4.18,4.35,0,"74,636,938"
19295,stellar,2013-04-28,4.30,4.40,4.18,4.35,0,"74,636,938"


## Create and Transform the Merged DataFrame

In [7]:
# Merge the two DataFrames on Date
stock_crypto_df = all_stock_df.merge(crypto_df,on=['Date'])

# Drop the old index of the stock DataFrame and set the new index to be Date
stock_crypto_df = stock_crypto_df.drop(columns=['Unnamed: 0'])

# Rename the columns to specify _Stock and _Crypto
stock_crypto_df.columns = stock_crypto_df.columns.str.replace('_x','_etf').str.replace('_y','_crypto')

# Extract the list of column names to reorder the columns 
col_list = list(stock_crypto_df.columns.values)
stock_index = col_list.index("ETF")
reordered_cols = [col_list[0]] + [col_list[stock_index]]+ col_list[1:(stock_index)]+ col_list[(stock_index+1):]
stock_crypto_df = stock_crypto_df[reordered_cols]

# Preview the merged dataframe with reordered columns
stock_crypto_df.head()

,Date,ETF,Open_etf,High_etf,Low_etf,Close_etf,Volume_etf,OpenInt,Currency,Open_crypto,High_crypto,Low_crypto,Close_crypto,Volume_crypto,Market Cap
0,2013-04-29,vti,74.952,75.393,74.842,75.227,2309512,0,bitcoin-sv,4.37,4.57,4.23,4.38,0,"75,388,964"
1,2013-04-29,vti,74.952,75.393,74.842,75.227,2309512,0,ethereum,134.44,147.49,134.00,144.54,0,"1,603,768,865"
2,2013-04-29,vti,74.952,75.393,74.842,75.227,2309512,0,bitcoin-cash,134.44,147.49,134.00,144.54,0,"1,603,768,865"
3,2013-04-29,vti,74.952,75.393,74.842,75.227,2309512,0,tezos,4.37,4.57,4.23,4.38,0,"75,388,964"
4,2013-04-29,vti,74.952,75.393,74.842,75.227,2309512,0,xrp,134.44,147.49,134.00,144.54,0,"1,603,768,865"


In [8]:
# Filter DataFrame to only show closing price of ETF and Cryptocurrency
stock_crypto_df_short = stock_crypto_df[['Date','ETF','Close_etf','Currency','Close_crypto']]

# Convert date to string for loading into SQL's date type
stock_crypto_df_short["Date"] = stock_crypto_df_short["Date"].dt.strftime('%Y-%m-%d')

# Rename columns to match SQL's lower case
stock_crypto_df_short = stock_crypto_df_short.rename(columns={'Date':'date_','ETF':'etf','Close_etf':'close_etf','Currency':'currency','Close_crypto':'close_crypto'})

# Convert closing price to type numeric (no thousands place commas)
stock_crypto_df_short['close_crypto'] = stock_crypto_df_short['close_crypto'].str.replace(',','')

# Preview final DataFrame
stock_crypto_df_short.head()

/Users/morganivey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,date_,etf,close_etf,currency,close_crypto
0,2013-04-29,vti,75.227,bitcoin-sv,4.38
1,2013-04-29,vti,75.227,ethereum,144.54
2,2013-04-29,vti,75.227,bitcoin-cash,144.54
3,2013-04-29,vti,75.227,tezos,4.38
4,2013-04-29,vti,75.227,xrp,144.54


In [9]:
# Save DataFrame to CSV file
stock_crypto_df_short.to_csv('Resources/condensed_ETF_crypto.csv')

In [15]:
len(stock_crypto_df_short)

16530388

## Load Transformed DataFrame to PostgreSQL

### Create database connection

In [11]:
# Connect to the database
connection_string = f'postgresql://{user_name}:{password}@{local_host}/stocks_db'
engine = create_engine(connection_string)

In [12]:
# List the table names in stocks_db database
engine.table_names()

['stocks_merged']

### Load DataFrames into database

In [13]:
stock_crypto_df_short.to_sql(name='stocks_merged', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the stock_crypto table

In [14]:
pd.read_sql_query('select * from stocks_merged', con=engine)

,date_,etf,close_etf,currency,close_crypto
0,2013-04-29,vti,75.227,bitcoin-sv,4.380000
1,2013-04-29,vti,75.227,ethereum,144.540000
2,2013-04-29,vti,75.227,bitcoin-cash,144.540000
3,2013-04-29,vti,75.227,tezos,4.380000
4,2013-04-29,vti,75.227,xrp,144.540000
5,2013-04-29,vti,75.227,bitcoin,144.540000
6,2013-04-29,vti,75.227,litecoin,4.380000
7,2013-04-29,vti,75.227,eos,4.380000
8,2013-04-29,vti,75.227,tether,144.540000
9,2013-04-29,vti,75.227,stellar,4.380000
